In [1]:
import os
import cv2
import numpy as np
from PIL import Image

In [2]:
BASE_PATH = "/home/varad/6422-DB-implementation/eva/eva/tutorials/blob-storage"
NPY_STORE_NAME = "video-blob.npy"
VIDEO_SRC = "/home/varad/6422-DB-implementation/eva/video/eva-video.mp4"

NPY_STORE = BASE_PATH + "/" + NPY_STORE_NAME

In [3]:
if not os.path.exists(os.path.dirname(NPY_STORE)):
    try:
        os.makedirs(os.path.dirname(NPY_STORE))
    except OSError as exc:
        # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [4]:
cap = cv2.VideoCapture(VIDEO_SRC)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [5]:
buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

In [6]:
# Manual append yeilds problematic results, np.save stores metadata for better retrieval mechanism.
# Should go with the recommended way to avoid compatibility issues further down the line
fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()

In [7]:
# this is time consuming, can be optimized with memmap??
# np memmaps for write are tricky: https://stackoverflow.com/questions/16149803/working-with-big-data-in-python-and-numpy-not-enough-ram-how-to-save-partial-r
with open(NPY_STORE, 'wb') as blobStore:
    np.save(blobStore, buf)

blobStore.close()

In [8]:
def get_nth_frame(n):
    img = np.memmap(NPY_STORE, dtype='uint8', mode='r+', shape=(frameHeight, frameWidth, 3), offset=n * frameHeight * frameWidth * 3)
    return img

In [9]:
def get_range_of_frames(start_range, end_range):
    img_range = np.memmap(NPY_STORE, dtype='uint8', mode='r+', shape=(end_range - start_range + 1, frameHeight, frameWidth, 3), offset= start_range * frameHeight * frameWidth * 3)
    return img_range

In [10]:
def show_np_array_img(img):
    img = cv2.cvtColor(img[...,::-1].copy(), cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img, 'RGB')
    img.show()

In [13]:
# Tests on a 15 second countdown video (500 * 1920 * 1020 * 3)
# img = get_nth_frame(120)
# show_np_array_img(img)
# Should show number 16 for n = 120

# img_range = get_range_of_frames(120, 125)
# for img in img_range:
#     show_np_array_img(img)
# Comment out to check if the number changes from 16 to 15 for start_range = 120 and end_range = 125